# [CS303]Practice 6 - BTS & Practice 7 - Improving BTS
In Practice 6 and 7, we will apply the BTS algorithm and the Improving BTS to solve the N-Queens problem.

In the following code, only two of the function usages are essential. One is `is_valid(self, state)`, which checks if the current state is legal; the other is `is_satisfy(self, state)`, which is to checks if the current board meets the win condition. 
The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.

In test block, for `Solver`,  `n` indicates the size of the test while `method` indicates which bts function will be used(`bts` or `improving_bts`); for method `solve`, `render` indicates whether to use a graphical interface representation.

## Practice 6 - BTS
You are required to implement complete the function `bts()`.
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. You may add any function you want. 
**Remember to test the case where N=6**.

📌 **DDL: Nov.9th**

## Practice 7 - Improving BTS
You are required to complete the function `improving_bts()`.
For `improving_bts()`, you can use one or more of the following strategies: Minimum Remaining Value, Least constraining value, and Forward checking. You should have a good performance (within 2s) **when N = 16 without GUI**. 

📌 **DDL: Nov.16th**

## Evaluation Criteria

Your submission will be assessed on:

* Clear understanding of every line of your code (no blind copy-paste).
* Successful compilation of the program.
* Correctness of the program logic.
* Reasonable efficiency of the solution.

### Grading Policy

* **On-time submission:** 60–100 points
* **Late submission:** 0 points


In [19]:
import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.N = n

    def is_valid(self, state):
        """
        check whether the state satisfies the condition or not.
        : param state: list of points (in (row id, col id) tuple form)
        : return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.N

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.N - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.N - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.N - i - 1, self.N - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.N, self.N], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.N + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.N + 1))

In [20]:
class Solver:
    def __init__(self, n, method):
        self.N = n
        self.problem = Problem(n)
        self.method = method

    def solve(self, render=True):
        if render:
            import pygame
            w, h = 90 * self.N + 10, 90 * self.N + 10
            screen = pygame.display.set_mode((w, h))
            screen.fill('white')
            action_generator = self.method(self.problem)
            clk = pygame.time.Clock()
            queen_img = pygame.image.load('./queen.png')
            while True:
                for event in pygame.event.get():
                    if event == pygame.QUIT:
                        exit()
                try:
                    actions = next(action_generator)
                    screen.fill('white')
                    for i in range(self.N + 1):
                        pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                        pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                    for action in actions:
                        i, j = action
                        screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                    pygame.display.flip()
                except StopIteration:
                    pass
                clk.tick(5)
            pass
        else:
            start_time = time.time()
            actions = None
            for actions in self.method(self.problem):
                pass
            if actions is not None:
                self.problem.print_state(actions)
                print(f"Solution found in {time.time() - start_time:.6f} seconds")
            else:
                print(f"No solution found in {time.time() - start_time:.6f} seconds")

# BTS: Backtracking search

In [21]:
def bts(problem):
    action_stack = []
    next_col = [0] * problem.N
    row = 0
    
    while not problem.is_satisfy(action_stack):
        if row < 0:
            return
        
        if next_col[row] >= problem.N:
            next_col[row] = 0
            row -= 1
            if row >= 0:
                action_stack.pop()
                next_col[row] += 1
            continue
        
        col = next_col[row]
        action_stack.append((row, col))
        
        if problem.is_valid(action_stack):
            row += 1
        else:
            action_stack.pop()
            next_col[row] += 1
            
    yield action_stack
    return

# Improving BTS 
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

## Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
## Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
## Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [22]:
def improving_bts(problem):
    action_stack = []  # 按时间顺序记录已分配的 (row, col)
    next_col = [0] * problem.N  # 每行下次尝试的索引（针对 ordered_cols）
    assigned = [None] * problem.N  # 当前每行的赋值或 None
    ordered_cols = [None] * problem.N  # 每行按 LCV 排序的候选列缓存

    def get_legal_columns(row_idx, current_state):
        legal = []
        for col in range(problem.N):
            if not any(r == row_idx for r, _ in current_state):
                test_state = current_state + [(row_idx, col)]
            else:
                # 如果 current_state 中已有该行的赋值，先移除再测试（通常不会发生）
                test_state = [p for p in current_state if p[0] != row_idx] + [(row_idx, col)]
            if problem.is_valid(test_state):
                legal.append(col)
        return legal

    def count_constraints(row_idx, col, current_state):
        test_state = current_state + [(row_idx, col)]
        if not problem.is_valid(test_state):
            return float('inf')
        constraints = 0
        for future_row in range(problem.N):
            if any(r == future_row for r, _ in test_state):
                continue
            for future_col in range(problem.N):
                future_test = test_state + [(future_row, future_col)]
                if not problem.is_valid(future_test):
                    constraints += 1
        return constraints

    def forward_check(current_state):
        # 确保所有未赋值行至少有一个合法列
        for check_row in range(problem.N):
            if any(r == check_row for r, _ in current_state):
                continue
            if not get_legal_columns(check_row, current_state):
                return False
        return True

    while True:
        # 完成解
        if problem.is_satisfy(action_stack):
            yield action_stack
            return

        # 前向检查失败 -> 回溯最近一次赋值
        if not forward_check(action_stack):
            if not action_stack:
                return
            last_r, _ = action_stack.pop()
            assigned[last_r] = None
            next_col[last_r] += 1
            # 清除缓存（状态变化需要重算）
            ordered_cols = [None] * problem.N
            continue

        # 选择未赋值变量：MRV（候选最少的行）
        unassigned_rows = [r for r in range(problem.N) if assigned[r] is None]
        if not unassigned_rows:
            # 没有未赋值但未满足 -> 无解或需回溯
            if not action_stack:
                return
            last_r, _ = action_stack.pop()
            assigned[last_r] = None
            next_col[last_r] += 1
            ordered_cols = [None] * problem.N
            continue

        # 计算每个未赋值行的合法列数，选择最小的（MRV）
        min_row = None
        min_len = None
        legal_map = {}
        for r in unassigned_rows:
            cols = get_legal_columns(r, action_stack)
            legal_map[r] = cols
            l = len(cols)
            if min_len is None or l < min_len:
                min_len = l
                min_row = r

        # 如果某行无合法列（应被 forward_check 捕获），这里也回溯以稳健处理
        if min_len == 0:
            if not action_stack:
                return
            last_r, _ = action_stack.pop()
            assigned[last_r] = None
            next_col[last_r] += 1
            ordered_cols = [None] * problem.N
            continue

        row = min_row
        # 为选中行准备按 LCV 排序的列列表
        if ordered_cols[row] is None:
            cols = legal_map.get(row, get_legal_columns(row, action_stack))
            cols.sort(key=lambda c: count_constraints(row, c, action_stack))
            ordered_cols[row] = cols

        cols = ordered_cols[row]
        # 如果该行的候选已尝试完 -> 回溯最近一次赋值
        if next_col[row] >= len(cols):
            next_col[row] = 0
            ordered_cols[row] = None
            if not action_stack:
                return
            last_r, _ = action_stack.pop()
            assigned[last_r] = None
            next_col[last_r] += 1
            ordered_cols = [None] * problem.N
            continue

        # 尝试给选中行赋值
        col = cols[next_col[row]]
        action_stack.append((row, col))
        assigned[row] = col

        # 如果有效且前向检查通过，继续（下次循环会重新选择 MRV）
        if problem.is_valid(action_stack) and forward_check(action_stack):
            # 不改变 next_col[row]（保留当前尝试索引），
            # 但为防止再次被误判为“已尝试完”，确保下一次重新选择时从 0 开始新行也 OK
            continue
        else:
            # 失败，撤销该赋值并尝试该行的下一个候选
            action_stack.pop()
            assigned[row] = None
            next_col[row] += 1
            ordered_cols[row] = None
            # 清除其它行缓存以确保一致性
            ordered_cols = [None] * problem.N
            continue

## test block

In [23]:
Solver(n=6, method=bts).solve(render=False)

_____________
|·|Q|·|·|·|·|
|·|·|·|Q|·|·|
|·|·|·|·|·|Q|
|Q|·|·|·|·|·|
|·|·|Q|·|·|·|
|·|·|·|·|Q|·|
-------------
Solution found in 0.029811 seconds


In [24]:
Solver(n=16, method=improving_bts).solve(render=False)

_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
---------------------------------
Solution found in 39.088660 seconds
